<a href="https://colab.research.google.com/github/adalbertii/LLM/blob/main/deepparse_test_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install deepparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.6/225.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.5/213.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 M

In [5]:
import pandas as pd

from deepparse import download_from_public_repository
from deepparse.dataset_container import PickleDatasetContainer
from deepparse.parser import AddressParser

In [9]:

address_parser = AddressParser(model_type="fasttext", device=0)

/usr/local/lib/python3.10/dist-packages/deepparse/parser/address_parser.py:1094: UserWarning: No CUDA device detected, device will be set to 'CPU'.
  warnings.warn("No CUDA device detected, device will be set to 'CPU'.", category=UserWarning)
/usr/local/lib/python3.10/dist-packages/deepparse/network/seq2seq.py:102: UserWarning: No pre-trained model where found in the cache directory /root/.cache/deepparse. Thus, we willautomatically download the pre-trained model.
  warnings.warn(


The fastText pretrained word embeddings will be downloaded (6.8 GO), this process will take several minutes.
(100.00%) [==================================================>]
Loading the embeddings model


In [10]:
# you can parse one address
parsed_address = address_parser("Kołaczyce , Mickiewicza 249, 23-200")

In [11]:
print(parsed_address)

The unparsed address is 'Kołaczyce , Mickiewicza 249, 23-200' and the parsed address is '('kołaczyce', 'Municipality') ('mickiewicza', 'StreetName') ('249', 'StreetNumber') ('23-200', 'PostalCode')'
